# Workshop 3: Explore NT scenario results, modify assumptions & benchmarking

:::{note} At the end of this notebook, you will be able to:

- Navigate and analyze NT scenario results
- Interpret current discrepancies between model outputs and the TYNDP 2024 NT scenario
- Compare 2030 and 2040 scenarios using PyPSA-Explorer
- Modify model assumptions and generate new scenario results
- Apply the benchmarking framework to identify areas for model improvement

:::

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install pypsa pypsa-explorer pandas matplotlib numpy
```
:::

In [ ]:
# uncomment for running this notebook on Colab
# !pip install pypsa pypsa-explorer pandas matplotlib numpy

In [ ]:
import os
from datetime import datetime
import pypsa
import zipfile
from urllib.request import urlretrieve
from pdf2image import convert_from_path
from pdf2image.exceptions import PDFPageCountError
import matplotlib.pyplot as plt
from pypsa_explorer import create_app
from pathlib import Path

pypsa.options.params.statistics.round = 3
pypsa.options.params.statistics.drop_zero = True
pypsa.options.params.statistics.nice_names = False
plt.rcParams["figure.figsize"] = [14, 7]

In [ ]:
def unzip_with_timestamps(zip_path, extract_to):
    """Unzip a file while preserving original file timestamps."""
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        for member in zip_ref.infolist():
            # Extract the file
            zip_ref.extract(member, extract_to)

            # Get the extracted file path
            extracted_path = os.path.join(extract_to, member.filename)

            # Get the modification time from the zip file
            date_time = datetime(*member.date_time)
            timestamp = date_time.timestamp()

            # Set both access and modification times
            os.utime(extracted_path, (timestamp, timestamp))

In [ ]:
urls = {
    "data/results.zip": "https://storage.googleapis.com/open-tyndp-data-store/workshop-03/results.zip",
}

os.makedirs("data", exist_ok=True)
os.makedirs("scripts", exist_ok=True)
for name, url in urls.items():
    if os.path.exists(name):
        print(f"File {name} already exists. Skipping download.")
    else:
        print(f"Retrieving {name} from GCP storage.")
        urlretrieve(url, name)
        print(f"File available in {name}.")

to_dir = "data/results"
if not os.path.exists(to_dir):
    print(f"Unzipping data/results.zip.")
    unzip_with_timestamps("data/results.zip", "data/results")
print(f"Open-TYNDP available in '{to_dir}'.")

print("Done")

# Interactive exploration with PyPSA-Explorer

PyPSA-Explorer is an interactive web dashboard that allows you to visualize and analyze energy system networks. It provides:
- Energy balance analysis with timeseries and aggregated views
- Capacity planning visualizations by carrier and region
- Economic analysis showing CAPEX/OPEX breakdowns
- Interactive geographical network maps
- Multi-network visualisation support

Let's load the NT scenario results and explore them using PyPSA-Explorer.

In [ ]:
# Load NT scenario networks for comparison
base_path = "data/results/results/networks/"

# Load networks directly into dictionary for PyPSA-Explorer
networks = {
    "NT 2030": pypsa.Network(base_path + "base_s_all___2030.nc"),
    "NT 2040": pypsa.Network(base_path + "base_s_all___2040.nc"),
}

The PyPSA-Explorer package can be started in two modes depending on your environment:

- **Local Jupyter**: Terminal command (**recommended**) or inline display
- **Google Colab**: The dashboard will launch inline, embedded directly in the notebook

Follow the instructions below based on your environment.

In [ ]:
# Detect if running on Google Colab
try:
    from google.colab import output

    IN_COLAB = True
    print(f"This notebook is running on Google Colab !")
except ImportError:
    IN_COLAB = False
    print(f"This notebook is running locally !")

port = 8050

## For Local Users

If running locally, the **recommended approach** is to launch PyPSA-Explorer from the terminal:

```bash
pypsa-explorer data/results/results/networks/base_s_all___2030.nc:NT_2030 data/results/results/networks/base_s_all___2040.nc:NT_2040
```

This will open the dashboard in your default browser at http://localhost:8050.

**Alternative**: The cell below can also launch the dashboard inline, but the terminal method is preferred for better performance.

In [ ]:
# Terminal method recommended
USE_TERMINAL = True  # Change to False if you want to launch inline display

if not IN_COLAB and not USE_TERMINAL:
    # Local Jupyter: Inline display
    app = create_app(networks)
    app.run(jupyter_mode="tab", port=port, debug=False)

## For Google Colab Users

If running on Google Colab, you need to start PyPSA-Explorer from the terminal before the iframe can display it.

Run the following command in the Google Colab terminal:

```python
pypsa-explorer data/results/results/networks/base_s_all___2030.nc:NT_2030 data/results/results/networks/base_s_all___2040.nc:NT_2040
```

In [ ]:
if IN_COLAB:
    # Google Colab: Use built-in Dash Jupyter support to display an iframe
    output.serve_kernel_port_as_iframe(port, height=1500)

**Tip for Colab users:** To view the dashboard in fullscreen mode, click the three dots (⋮) in the top-right corner of the output cell and select **"View output fullscreen"**.

## Using the Dashboard

Once the dashboard opens, you can:

1. **Energy Balance Tab**: View production, consumption, and storage patterns
   - Switch between timeseries and aggregated views
   - Filter by energy carrier (electricity, hydrogen, etc.)
   - Filter by country/region

2. **Capacity Tab**: Analyze installed capacities
   - Compare capacity buildout between 2030 and 2040 scenarios
   - View by technology type and region
   - Identify discrepancies with TYNDP 2024 targets

3. **Economics Tab**: Examine costs and revenues
   - CAPEX and OPEX breakdowns by technology
   - Regional cost comparisons
   - Investment requirements

4. **Network Map**: Visualize the geographical network
   - Interactive map with buses, lines, and generators
   - Zoom and pan to explore specific regions
   - View transmission constraints

**Tip:** Use the scenario selector dropdown to switch between NT 2030 and NT 2040 for direct comparison.

# Benchmark results

In Workshop 2, we introduced a benchmarking framework to systematically compare Open-TYNDP model outputs against TYNDP 2024. This framework helps us:

- **Identify discrepancies**: Compare demands, installed capacities and generation volumes between model results and TYNDP targets
- **Quantify differences**: Calculate deviations by technology, country, and investment year
- **Guide improvements**: Prioritize areas where the model requires improvements

Let's apply this framework to our NT 2030 and 2040 scenario results to understand where the current model implementation aligns with or diverges from TYNDP expectations.

To do so, let's first define a function to display benchmarks.

In [ ]:
def show_benchmarks(
    fn: str,
    years:list = [2030, 2040],
    bench_path: str = "data/results/results/validation/graphics_s_all___all_years",
):
    try:
        images = [
            convert_from_path(Path(bench_path, f"{fn}_{y}.pdf"))[0]
            for y in years
        ]
    except PDFPageCountError:
        print("File not found, skipping...")
        return

    fig, axes = plt.subplots(1, 2)
    for ax, img in zip(axes, images):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

## Overview

We start with an overview of the sMAPE across all carriers. This provides a high-level understanding of the magnitude of the error across all benchmarks, carriers and planning horizons. It can clearly be observed that some features were significantly improved since our introduction of the framework. However, there is also still room for improvement.

In [ ]:
display(
    convert_from_path(Path("data/results/results/validation/kpis_eu27_s_all___all_years.pdf"))[0]
)

## Final Energy Demand

Let's now examine each benchmark category in detail, starting with final energy demand across all carriers.

In [ ]:
show_benchmarks("benchmark_final_energy_demand_eu27_cy2009")

## Electricity Demand

Electricity demand is an exogenous input to the model, directly taken from TYNDP 2024 data. Therefore, we get a perfect match between the model results and TYNDP 2024.

In [ ]:
show_benchmarks("benchmark_elec_demand_eu27_cy2009")

## Hydrogen Demand

Hydrogen demand is mainly defined exogenously. Therefore, we expect a good match between the model results and TYNDP 2024. The electricity generation, on the other hand, is endogenous.

In [ ]:
show_benchmarks("benchmark_hydrogen_demand_eu27_cy2009")

## Methane Demand

Methane demand includes exogenous consumption, as well as endogenous demand for power generation and SMR (steam methane reforming).

In [ ]:
show_benchmarks("benchmark_methane_demand_eu27_cy2009")

## Power Generation Capacities

Installed generation capacities are now converging towards the values of TYNDP 2024. Both renewable and conventional capacities are now fully integrated from the PEMMDB.

In [ ]:
show_benchmarks("benchmark_power_capacity_eu27_cy2009")

## Electricity Generation

Actual electricity generation differs slightly from the TYNDP 2024 generation mix. However, the total generation values are close.

In [ ]:
show_benchmarks("benchmark_power_generation_eu27_cy2009")

## Hydrogen Supply

TODO

In [ ]:
show_benchmarks("benchmark_hydrogen_supply_eu27_cy2009")

## Methane Supply

TODO

In [ ]:
show_benchmarks("benchmark_methane_supply_eu27_cy2009")

## Biomass Supply

TODO

In [ ]:
show_benchmarks("benchmark_biomass_supply_eu27_cy2009")

## Energy Imports

Energy imports include hydrogen, methane, liquids and solids. Currently, the model cannot import biomass.

In [ ]:
show_benchmarks("benchmark_energy_imports_eu27_cy2009")

# Modify assumptions

# Solutions

## Task 1: # TODO